## 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 인간라벨링 데이터 로드

In [ ]:
# 사람이 직접 매긴 ESG 점수 데이터 로드
human_esg_scores = pd.read_csv('/content/drive/MyDrive/merged_df.csv')
human_esg_scores.rename(columns={'Column1': 'Sentence'}, inplace=True)
human_esg_scores

### 디폴트 모델 생성 라벨데이터 전처리

In [ ]:
# 프로그램으로 생성한 ESG 점수 데이터
import pandas as pd
# 예시 데이터를 기반으로 데이터프레임 생성
data = pd.read_csv('/content/drive/MyDrive/esg_scores_final.csv')
# 데이터프레임 생성
df = pd.DataFrame(data)
# First, let's extract the 'Sentence' and 'ESG Score' columns
df2 = df[['Sentence', 'ESG Score']]
# E, S, G 값을 추출하여 새로운 열에 저장
df2['ESG Score'] = df2['ESG Score'].apply(lambda x: x[1:-1])
# 'ESG Score' 컬럼의 값을 쉼표(,)로 분리하고, expand=True 옵션을 사용하여 분리된 값을 별도의 컬럼으로 확장합니다.
df2[['E', 'S', 'G']] = df2['ESG Score'].str.split(',', expand=True)
df2.drop('ESG Score', axis=1, inplace=True)
# 문자열 형태의 숫자를 정수형으로 변환합니다. 필요한 경우 astype을 사용하여 데이터 타입을 변환할 수 있습니다.
df2['E'] = df2['E'].astype(int)
df2['S'] = df2['S'].astype(int)
df2['G'] = df2['G'].astype(int)
generated_esg_scores = df2
generated_esg_scores

### 파인튜닝모델 라벨데이터 전처리

In [ ]:
# 파인튜닝된 모델로 생성한 ESG 점수 데이터
import pandas as pd
# 예시 데이터를 기반으로 데이터프레임 생성
data = pd.read_csv('/content/drive/MyDrive/ft_esg_scores_final.csv')
# 데이터프레임 생성
df = pd.DataFrame(data)
# First, let's extract the 'Sentence' and 'ESG Score' columns
df2 = df[['Sentence', 'ESG Score']]
# E, S, G 값을 추출하여 새로운 열에 저장
df2['ESG Score'] = df2['ESG Score'].apply(lambda x: x[1:-1])
# 'ESG Score' 컬럼의 값을 쉼표(,)로 분리하고, expand=True 옵션을 사용하여 분리된 값을 별도의 컬럼으로 확장합니다.
df2[['E', 'S', 'G']] = df2['ESG Score'].str.split(',', expand=True)
df2.drop('ESG Score', axis=1, inplace=True)
# 문자열 형태의 숫자를 정수형으로 변환합니다. 필요한 경우 astype을 사용하여 데이터 타입을 변환할 수 있습니다.
df2['E'] = df2['E'].astype(int)
df2['S'] = df2['S'].astype(int)
df2['G'] = df2['G'].astype(int)
ft_gen_esg_scores = df2
ft_gen_esg_scores

### 긍부정 단일화 (선택)

In [ ]:
# Function to replace values
def replace_values(value):
    if value == -2:
        return -1
    elif value == 2:
        return 1
    else:
        return value

df = ft_gen_esg_scores
# Applying the function to each column (E, S, G)
df["E"] = df["E"].apply(replace_values)
df["S"] = df["S"].apply(replace_values)
df["G"] = df["G"].apply(replace_values)
ft_gen_esg_scores = df

df = generated_esg_scores
# Applying the function to each column (E, S, G)
df["E"] = df["E"].apply(replace_values)
df["S"] = df["S"].apply(replace_values)
df["G"] = df["G"].apply(replace_values)
generated_esg_scores = df

df = human_esg_scores
# Applying the function to each column (E, S, G)
df["E"] = df["E"].apply(replace_values)
df["S"] = df["S"].apply(replace_values)
df["G"] = df["G"].apply(replace_values)
human_esg_scores = df

## 디폴트 모델과 인간라벨링 비교

In [ ]:
# 비교를 위해 두 데이터프레임을 병합합니다.
comparison_df = pd.merge(generated_esg_scores, human_esg_scores, on='Sentence', suffixes=('_generated', '_human'))

# E, S, G 각 요소에 대한 일치 여부를 새로운 컬럼에 추가합니다.
comparison_df['E_Match'] = comparison_df['E_generated'] == comparison_df['E_human']
comparison_df['S_Match'] = comparison_df['S_generated'] == comparison_df['S_human']
comparison_df['G_Match'] = comparison_df['G_generated'] == comparison_df['G_human']

Sentencecount = comparison_df['Sentence'].nunique()
# 모든 요소가 일치하는 문서의 수를 카운트합니다.
print("디폴트 모델과 인간라벨링 비교")
all_match_count = comparison_df[comparison_df[['E_Match', 'S_Match', 'G_Match']].all(axis=1)].shape[0]
all_match_counte = comparison_df[comparison_df[['E_Match']].all(axis=1)].shape[0]
all_match_counts = comparison_df[comparison_df[['S_Match']].all(axis=1)].shape[0]
all_match_countg = comparison_df[comparison_df[['G_Match']].all(axis=1)].shape[0]
print(f"ESG scores (All sectors) match: {all_match_count}/{Sentencecount}")
print(f"ESG scores (sector 'E') match: {all_match_counte}/{Sentencecount}")
print(f"ESG scores (sector 'S') match: {all_match_counts}/{Sentencecount}")
print(f"ESG scores (sector 'G') match: {all_match_countg}/{Sentencecount}")
print(f"모두일치: {all_match_count/Sentencecount*100} %")
print(f"E만 일치: {all_match_counte/Sentencecount*100} %")
print(f"S만 일치: {all_match_counts/Sentencecount*100} %")
print(f"G만 일치: {all_match_countg/Sentencecount*100} %")

디폴트 모델과 인간라벨링 비교
ESG scores (All sectors) match: 89/913
ESG scores (sector 'E') match: 615/913
ESG scores (sector 'S') match: 381/913
ESG scores (sector 'G') match: 302/913
모두일치: 9.748083242059145 %
E만 일치: 67.36035049288061 %
S만 일치: 41.73055859802848 %
G만 일치: 33.07776560788609 %


## 파인튜닝모델과 라벨링 모델 비교

In [ ]:
# 비교를 위해 두 데이터프레임을 병합합니다. (사람 -파인)
comparison_df = pd.merge(ft_gen_esg_scores, human_esg_scores, on='Sentence', suffixes=('_ftmodel', '_human'))

# E, S, G 각 요소에 대한 일치 여부를 새로운 컬럼에 추가합니다.
comparison_df['E_Match'] = comparison_df['E_ftmodel'] == comparison_df['E_human']
comparison_df['S_Match'] = comparison_df['S_ftmodel'] == comparison_df['S_human']
comparison_df['G_Match'] = comparison_df['G_ftmodel'] == comparison_df['G_human']

In [ ]:
Sentencecount = comparison_df['Sentence'].nunique()
# 모든 요소가 일치하는 문서의 수를 카운트합니다.
all_match_count = comparison_df[comparison_df[['E_Match', 'S_Match', 'G_Match']].all(axis=1)].shape[0]
all_match_counte = comparison_df[comparison_df[['E_Match']].all(axis=1)].shape[0]
all_match_counts = comparison_df[comparison_df[['S_Match']].all(axis=1)].shape[0]
all_match_countg = comparison_df[comparison_df[['G_Match']].all(axis=1)].shape[0]
print("파인튜닝모델과 인간라벨링 비교")
print(f"ESG scores (All sectors) match: {all_match_count}/{Sentencecount}")
print(f"ESG scores (sector 'E') match: {all_match_counte}/{Sentencecount}")
print(f"ESG scores (sector 'S') match: {all_match_counts}/{Sentencecount}")
print(f"ESG scores (sector 'G') match: {all_match_countg}/{Sentencecount}")
print(f"모두일치: {all_match_count/Sentencecount*100} %")
print(f"E만 일치: {all_match_counte/Sentencecount*100} %")
print(f"S만 일치: {all_match_counts/Sentencecount*100} %")
print(f"G만 일치: {all_match_countg/Sentencecount*100} %")

파인튜닝모델과 인간라벨링 비교
ESG scores (All sectors) match: 103/914
ESG scores (sector 'E') match: 655/914
ESG scores (sector 'S') match: 420/914
ESG scores (sector 'G') match: 294/914
모두일치: 11.269146608315099 %
E만 일치: 71.66301969365426 %
S만 일치: 45.951859956236326 %
G만 일치: 32.16630196936542 %


## 파인튜닝모델과 디폴트모델 비교

In [ ]:
# 비교를 위해 두 데이터프레임을 병합합니다. (파인튜닝모델과 기본모델 비교)
comparison_df = pd.merge(ft_gen_esg_scores, generated_esg_scores, on='Sentence', suffixes=('_ftmodel', '_model'))

# E, S, G 각 요소에 대한 일치 여부를 새로운 컬럼에 추가합니다.
comparison_df['E_Match'] = comparison_df['E_ftmodel'] == comparison_df['E_model']
comparison_df['S_Match'] = comparison_df['S_ftmodel'] == comparison_df['S_model']
comparison_df['G_Match'] = comparison_df['G_ftmodel'] == comparison_df['G_model']

Sentencecount = comparison_df['Sentence'].nunique()
# 모든 요소가 일치하는 문서의 수를 카운트합니다.
all_match_count = comparison_df[comparison_df[['E_Match', 'S_Match', 'G_Match']].all(axis=1)].shape[0]
all_match_counte = comparison_df[comparison_df[['E_Match']].all(axis=1)].shape[0]
all_match_counts = comparison_df[comparison_df[['S_Match']].all(axis=1)].shape[0]
all_match_countg = comparison_df[comparison_df[['G_Match']].all(axis=1)].shape[0]
print("파인튜닝모델과 디폴트모델 비교")
print(f"ESG scores (All sectors) match: {all_match_count}/{Sentencecount}")
print(f"ESG scores (sector 'E') match: {all_match_counte}/{Sentencecount}")
print(f"ESG scores (sector 'S') match: {all_match_counts}/{Sentencecount}")
print(f"ESG scores (sector 'G') match: {all_match_countg}/{Sentencecount}")
print(f"모두일치: {all_match_count/Sentencecount*100} %")
print(f"E만 일치: {all_match_counte/Sentencecount*100} %")
print(f"S만 일치: {all_match_counts/Sentencecount*100} %")
print(f"G만 일치: {all_match_countg/Sentencecount*100} %")

파인튜닝모델과 디폴트모델 비교
ESG scores (All sectors) match: 1044/1570
ESG scores (sector 'E') match: 1419/1570
ESG scores (sector 'S') match: 1397/1570
ESG scores (sector 'G') match: 1354/1570
모두일치: 66.4968152866242 %
E만 일치: 90.38216560509554 %
S만 일치: 88.98089171974523 %
G만 일치: 86.24203821656052 %
